In [ ]:
#CUHK 2021-2022 Term 1 
#FETC5510 Group 6 Project

#Dataset can be found in here (https://www.kaggle.com/c/santander-product-recommendation/data)
#The following codes are used to build a product recommendation system based on the customer historical data.
#The recommendation system involves two filtering approaches: collaborative filtering and demographic filtering
#Output: 
#- Combined demographic-based and memory-based probabilities
#- Top 1 recommended product for each customer

In [1]:
#Step 0: Module installing before building predictive modeling

import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install psutil
!{sys.executable} -m pip install prettytable 

In [1]:
#Step 0: Module importing before building predictive modeling

import numpy as np
import pandas as pd
import csv
import datetime
from operator import sub
from sklearn import preprocessing, ensemble, metrics
import os
import gc
import psutil
import math
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from scipy.spatial.distance import pdist, wminkowski, squareform
from prettytable import PrettyTable 

pd.options.display.max_rows = 100
pd.options.display.max_columns = None


In [2]:
#Step 0: Data importing
#Please note that there should have two set of datasets in the directory before running this code

path = '../FETC5510/'
traindat = pd.read_csv(path + 'training set.csv', low_memory = True)
testdat = pd.read_csv(path + 'testing set.csv', low_memory = True)

/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
#[Other than modeling] Check the number of records in training set and testing set

print("{} {}".format("Number of Record in Training Set: ",len(traindat)))
print("{} {}".format("Number of Record in Testing Set: ",len(testdat)))

Number of Record in Training Set:  13647309
Number of Record in Testing Set:  929615


In [20]:
#[Other than modeling] Summarize the counting of records in training set by date

traindat['fecha_dato'].value_counts().sort_index()

2015-01-28    625457
2015-02-28    627394
2015-03-28    629209
2015-04-28    630367
2015-05-28    631957
2015-06-28    632110
2015-07-28    829817
2015-08-28    843201
2015-09-28    865440
2015-10-28    892251
2015-11-28    906109
2015-12-28    912021
2016-01-28    916269
2016-02-28    920904
2016-03-28    925076
2016-04-28    928274
2016-05-28    931453
Name: fecha_dato, dtype: int64

In [21]:
#[Other than modeling] Summarize the counting of records in testing set by date

testdat['fecha_dato'].value_counts().sort_index()

2016-06-28    929615
Name: fecha_dato, dtype: int64

In [3]:
#Step 1: Data cleaning - Dropping columns deemed surplus to requirements

#Define columns to be used for modeling (i.e. demographic data and product data)

#Columns of Demographic Data
demographic_cols = ['fecha_dato',
 'ncodpers','ind_empleado','pais_residencia','sexo','age','fecha_alta','ind_nuevo','antiguedad','indrel',
 'indrel_1mes','tiprel_1mes','indresi','indext','canal_entrada','indfall',
 'tipodom','cod_prov','ind_actividad_cliente','renta','segmento']

#Columns which no longer required
notuse = ["ult_fec_cli_1t","nomprov"]

#Columns of Product
product_col = [
 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1',
 'ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1',
 'ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
 'ind_nom_pens_ult1','ind_recibo_ult1']

#Combine demographic columns and product columns and store them into a variable
train_cols = demographic_cols + product_col

#Create trimmed datasets (only containing necessary columns)
traindat = traindat.filter(train_cols)
testdat  = testdat.filter(train_cols)


In [4]:
#[Other than modeling] Identify the columns with missing data

traindat.isnull().sum()

fecha_dato                     0
ncodpers                       0
ind_empleado               27734
pais_residencia            27734
sexo                       27804
age                            0
fecha_alta                 27734
ind_nuevo                  27734
antiguedad                     0
indrel                     27734
indrel_1mes               149781
tiprel_1mes               149781
indresi                    27734
indext                     27734
canal_entrada             186126
indfall                    27734
tipodom                    27735
cod_prov                   93591
ind_actividad_cliente      27734
renta                    2794375
segmento                  189368
ind_ahor_fin_ult1              0
ind_aval_fin_ult1              0
ind_cco_fin_ult1               0
ind_cder_fin_ult1              0
ind_cno_fin_ult1               0
ind_ctju_fin_ult1              0
ind_ctma_fin_ult1              0
ind_ctop_fin_ult1              0
ind_ctpp_fin_ult1              0
ind_deco_f

In [5]:
#Step 1: Data cleaning - Handling with missing value
#1. Factor variables: either impute the most common factor level or set to a new 'missing' level
#2. Numerical variables: set the missing value equal to the average for each province
#3. Product variables: set to 0

traindat.age = pd.to_numeric(traindat.age, errors='coerce')
traindat.renta = pd.to_numeric(traindat.renta, errors='coerce')
traindat.antiguedad = pd.to_numeric(traindat.antiguedad, errors='coerce')

traindat.loc[traindat['ind_empleado'].isnull(),'ind_empleado'] = 'N'
traindat.loc[traindat['pais_residencia'].isnull(),'pais_residencia'] = 'ES'
traindat.loc[traindat['sexo'].isnull(),'sexo'] = 'V'
traindat.fecha_alta = traindat.fecha_alta.astype('datetime64[ns]')
traindat.loc[traindat['fecha_alta'].isnull(), 'fecha_alta'] = pd.Timestamp(2011,9,1)
traindat.loc[traindat['ind_nuevo'].isnull(), 'ind_nuevo'] = 0
traindat.loc[traindat['indrel'].isnull(), 'indrel'] = 1
traindat.indrel_1mes = traindat.indrel_1mes.astype('str').str.slice(0,1)
traindat.loc[traindat['indrel_1mes'].isnull(), 'indrel_1mes'] = '1'
traindat.loc[traindat['tiprel_1mes'].isnull(), 'tiprel_1mes'] = 'I'
traindat.loc[traindat['indresi'].isnull(), 'indresi'] = 'S'
traindat.loc[traindat['indext'].isnull(), 'indext'] = 'N'
traindat.loc[traindat['canal_entrada'].isnull(), 'canal_entrada'] = 'MIS'
traindat.loc[traindat['indfall'].isnull(), 'indfall'] = 'N'
traindat.loc[traindat['tipodom'].isnull(), 'tipodom'] = 0.0
traindat.loc[traindat['cod_prov'].isnull(), 'cod_prov'] = 28.0
traindat.loc[traindat['ind_actividad_cliente'].isnull(), 'ind_actividad_cliente'] = 0.0
traindat["renta"] = traindat[['renta','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace renta with provincial mean
traindat["age"] = traindat[['age','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace age with provincial mean
traindat["antiguedad"] = traindat[['antiguedad','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace antiguedad with provincial mean
traindat.loc[traindat['segmento'].isnull(), 'segmento'] = '02 - PARTICULARES'
traindat.loc[traindat['ind_nomina_ult1'].isnull(), 'ind_nomina_ult1'] = 0
traindat.loc[traindat['ind_nom_pens_ult1'].isnull(), 'ind_nom_pens_ult1'] = 0

#Impute test data

testdat.age = pd.to_numeric(testdat.age, errors='coerce')
testdat.antiguedad = pd.to_numeric(testdat.antiguedad, errors='coerce')
testdat.renta = pd.to_numeric(testdat.renta, errors='coerce')

testdat.loc[testdat['sexo'].isnull(),'sexo'] = 'V'
testdat.indrel_1mes = testdat.indrel_1mes.astype('str').str.slice(0,1)
testdat.loc[testdat['indrel_1mes'].isnull(), 'indrel_1mes'] = '1'
testdat.loc[testdat['tiprel_1mes'].isnull(), 'tiprel_1mes'] = 'I'
testdat.loc[testdat['canal_entrada'].isnull(), 'canal_entrada'] = 'MIS'
testdat.loc[testdat['cod_prov'].isnull(), 'cod_prov'] = 28.0
testdat.loc[testdat['segmento'].isnull(), 'segmento'] = '02 - PARTICULARES'
testdat["renta"] = testdat[['renta','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace renta with provincial mean
testdat["age"] = testdat[['age','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace age with provincial mean
testdat["antiguedad"] = testdat[['antiguedad','cod_prov']].groupby("cod_prov").transform(lambda x: x.fillna(x.mean())) #Replace antiguedad with provincial mean

In [6]:
#[Other than modeling] Check to make sure all missing data has been filled

traindat.isnull().sum()

fecha_dato               0
ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
fecha_alta               0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
tipodom                  0
cod_prov                 0
ind_actividad_cliente    0
renta                    0
segmento                 0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
i

In [7]:
#Step 2: Feature engineering
#Bin the continuous variables and mutate product ownership variables

#Additional data cleaning
#Observation: based on (omitted) EDA, a pre/post 2011 split would make sense for fecha_alta; as credit recovered following the 2008 crash, we may expect to see different user types
traindat["fecha_alta"] = traindat["fecha_alta"].astype("datetime64")
testdat["fecha_alta"] = testdat["fecha_alta"].astype("datetime64")

#Observation: on a log scale, the salary data is broadly normal. We can take low-medium-high bounds using quartiles
traindat["renta"] = np.log(traindat["renta"])
testdat["renta"] = np.log(testdat["renta"])

#Bin the continuous variables
bins_dt = pd.date_range('1994-01-01', freq='16Y', periods=3)
bins_str = bins_dt.astype(str).values
labels = ['({}, {}]'.format(bins_str[i-1], bins_str[i]) for i in range(1, len(bins_str))]

traindat['fecha_alta'] = pd.cut(traindat.fecha_alta.astype(np.int64)//10**9,
                   bins=bins_dt.astype(np.int64)//10**9,
                   labels=labels)

testdat['fecha_alta'] = pd.cut(testdat.fecha_alta.astype(np.int64)//10**9,
                   bins=bins_dt.astype(np.int64)//10**9,
                   labels=labels)


bins_renta = [0,np.percentile(traindat.renta, 25),np.percentile(traindat.renta, 75),25]

traindat['renta'] = pd.cut(traindat.renta,
                   bins=bins_renta)

testdat['renta'] = pd.cut(testdat.renta,
                   bins=bins_renta)


bins_age = [0,25,42,60,1000]
labels_age = ['young','middle','older','old']

traindat['age'] = pd.cut(traindat.age,
                   bins=bins_age,
                   labels=labels_age)

testdat['age'] = pd.cut(testdat.age,
                   bins=bins_age,
                   labels=labels_age)


bins_anti = [-1,220,300]
labels_anti = ['new','old']

#Remove negative antiguedad values
traindat.antiguedad[traindat.antiguedad<0] = 0

traindat['antiguedad'] = pd.cut(traindat.antiguedad,
                   bins=bins_anti,
                   labels=labels_anti)

testdat['antiguedad'] = pd.cut(testdat.antiguedad,
                   bins=bins_anti,
                   labels=labels_anti)

/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3095308280.py:21: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  traindat['fecha_alta'] = pd.cut(traindat.fecha_alta.astype(np.int64)//10**9,
/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3095308280.py:22: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  bins=bins_dt.astype(np.int64)//10**9,
/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3095308280.py:25: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  testdat['fecha_alta'] = pd.cut(testdat.fecha_alta.astype(np.int64)//10**9,
/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3095308280.py:26: FutureWarning: casting datetime64[ns] valu

In [8]:
#Step 3: Data Subsetting
#Only used June 2015 data (i.e. one year before the test data), its associated lagged month and lagged May 2016 as a predictor

traindat = traindat[traindat.fecha_dato.isin(['2015-05-28','2015-06-28','2016-05-28'])]

In [9]:
#In order to join each user with itself in the previous month

#First sort data based on key columns
traindat = traindat.sort_values(['ncodpers','fecha_dato'],ascending=[True,True]).reset_index(drop=True)
print('sort completed')

#Then create a new dataset where the index is incremented...
traindat['new'] = traindat.index
train_index = traindat.copy()
train_index['new'] += 1

#Then merge the dataset with itself to add each user's purchases in the previous month
#Rename these new columns with a '_previous' suffix 
merge_drop_cols = demographic_cols.copy()
merge_drop_cols.remove('ncodpers')
traindat_use = pd.merge(traindat,train_index.drop(merge_drop_cols,1), on=['new','ncodpers'],how='left',suffixes=['','_previous'])
print('merge completed')

#Replace current with (current - previous) to obtain purchase indicators
for i in product_col:
    traindat_use[i] = traindat_use[i]-traindat_use[i+"_previous"]
    
    #Replace negative values with 0: 
    #if a user gets rid of a product from month x to month x+1, this registers as no purchase in the evaluation metric, so treat it as no purchase made
    traindat_use[i][traindat_use[i] < 0] = 0

#Fill in na values created by merge
traindat_use[product_col] = traindat_use[product_col].fillna(0)
new_product_col = [i + "_previous" for i in product_col]
traindat_use[new_product_col] = traindat_use[new_product_col].fillna(0)

#Delete redundant objects to free up memory
del train_index

sort completed


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/20090700.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  traindat_use = pd.merge(traindat,train_index.drop(merge_drop_cols,1), on=['new','ncodpers'],how='left',suffixes=['','_previous'])


merge completed


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/20090700.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindat_use[i][traindat_use[i] < 0] = 0


In [10]:
#Add purchase history columns to the testing set to prediction purpose
test_col = product_col + ['ncodpers']
testdat_use = pd.merge(testdat,traindat[traindat.fecha_dato=='2016-05-28'][test_col],on='ncodpers',how='left',suffixes=['','_previous'])

testdat_use.rename(
    columns={i:j for i,j in zip(product_col,new_product_col)}, inplace=True
)

testdat_use[new_product_col] = testdat_use[new_product_col].fillna(0)

#Delete redundant objects to free up memory
del traindat, testdat
gc.collect()

736

In [11]:
#Step 4: De-duplicate the Data

#Pull through variables for memory-based CF
traindat_purchases = traindat_use[traindat_use.fecha_dato == '2015-06-28'][product_col].copy()
traindat_final = traindat_use[traindat_use.fecha_dato == '2015-06-28'][new_product_col].copy()

#Pull through variables for demographic-based CF
demog_col = ['sexo','age','fecha_alta','ind_nuevo','indrel','indresi','indfall','tipodom','ind_actividad_cliente']
traindat_demog_final = traindat_use[traindat_use.fecha_dato == '2015-06-28'][demog_col].copy()

#Transform demographic factor variables into binary format
sexo_map = {'V': 1,'H': 0}
age_map = {'old': 1,'young': 0}
fecha_alta_map = {'(1994-12-31, 2010-12-31]': 1,'(2010-12-31, 2026-12-31]': 0}
indresi_map = {'S': 1,'N': 0}
indfall_map = {'S': 1,'N': 0}

traindat_demog_final.loc[traindat_demog_final['age']=='older', 'age'] = 'old'
traindat_demog_final.loc[traindat_demog_final['age']=='middle', 'age'] = 'young'
traindat_demog_final.sexo = [sexo_map[item] for item in traindat_demog_final.sexo]
traindat_demog_final.age = [age_map[item] for item in traindat_demog_final.age]
traindat_demog_final.fecha_alta = [fecha_alta_map[item] for item in traindat_demog_final.fecha_alta]
traindat_demog_final.indresi = [indresi_map[item] for item in traindat_demog_final.indresi]
traindat_demog_final.indfall = [indfall_map[item] for item in traindat_demog_final.indfall]

#Collect all the observed combinations of purchase history
new_product_col_aug = new_product_col + ['ncodpers']
testdat_final = testdat_use[new_product_col_aug].copy()
testdat_final_unique = testdat_final.drop('ncodpers',1).drop_duplicates().copy().reset_index(drop=True)

#Transform demographic factor variables into binary format
demog_col_aug = demog_col + ['ncodpers']
testdat_demog_final = testdat_use[demog_col_aug].copy()

testdat_demog_final.loc[testdat_demog_final['age']=='older', 'age'] = 'old'
testdat_demog_final.loc[testdat_demog_final['age']=='middle', 'age'] = 'young'
testdat_demog_final.sexo = [sexo_map[item] for item in testdat_demog_final.sexo]
testdat_demog_final.age = [age_map[item] for item in testdat_demog_final.age]
testdat_demog_final.fecha_alta = [fecha_alta_map[item] for item in testdat_demog_final.fecha_alta]
testdat_demog_final.indresi = [indresi_map[item] for item in testdat_demog_final.indresi]
testdat_demog_final.indfall = [indfall_map[item] for item in testdat_demog_final.indfall] 

testdat_demog_final_unique = testdat_demog_final.drop('ncodpers',1).drop_duplicates().copy().reset_index(drop=True)

/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/1191215837.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  testdat_final_unique = testdat_final.drop('ncodpers',1).drop_duplicates().copy().reset_index(drop=True)
/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/1191215837.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  testdat_demog_final_unique = testdat_demog_final.drop('ncodpers',1).drop_duplicates().copy().reset_index(drop=True)


In [12]:
#Split the training data into 'training' and 'test' sets

#Create 80% index
traindat_index = np.random.rand(len(traindat_final)) < 0.8

#Create traindat_train
traindat_train = traindat_final[traindat_index]

#Create traindat_test
traindat_test = traindat_final[~traindat_index]

#Make traindat_test unique
traindat_test_unique = traindat_test.drop_duplicates().copy().reset_index(drop=True)

#Create traindat_purchases
traindat_purchases_train = traindat_purchases[traindat_index]

#Create traindat_purchases_test for verification
traindat_purchases_test = traindat_purchases[~traindat_index]

#Create training ncodpers index
traindat_ncodpers = traindat_use[traindat_use.fecha_dato == '2015-06-28'][traindat_index][['fecha_dato','ncodpers']]
traindat_test_ncodpers = traindat_use[traindat_use.fecha_dato == '2015-06-28'][~traindat_index][['fecha_dato','ncodpers']]

#Repeat for demographic columns
#Create traindat_demog_train
traindat_demog_train = traindat_demog_final[traindat_index]

#Create traindat_demog_test
traindat_demog_test = traindat_demog_final[~traindat_index]

#Make traindat_demog_test unique
traindat_demog_test_unique = traindat_demog_test.drop_duplicates().copy().reset_index(drop=True)

In [13]:
#Step 5: Build Demographic-Based and Memory-Based Similarity Matrices

predict_product_col = [i + "_predict" for i in new_product_col]

def probability_calculation(dataset,training,training_purchases,used_columns,metric,test_remap,print_option=False):
    #'dataset' takes the unique test data with purchase/demographic history; 'training' are the training data that we calculate distances to
    
    n = dataset.shape[0]
    
    for index, row in dataset.iterrows():
        if print_option == True:
            print(str(index) + '/' + str(n))
        row_use = row.to_frame().T
        
        #Store purchase history for the test users
        row_history = row_use[used_columns]
        
        #Calculate distances between the test point and each training point based on selected binary features
        #Use 'manhattan' when data was binary - when weighted against demographics, use Euclidean
        distances = metrics.pairwise_distances(row_use,training,metric=metric) + 1e-6
        
        #Normalise distances: previously used 24-distances, and 1/(1+distances), but the asymptotic behaviour of 1/distances gives the most accurate predictions.
        norm_distances = 1/distances
        
        #Take dot product between distance to training point and training point's purchase history to obtain ownership likelihood matrix
        sim = pd.DataFrame(norm_distances.dot(training_purchases)/np.sum(norm_distances),columns = new_product_col)
        if(index == 0):
            probabilities = sim
        else:
            probabilities = probabilities.append(sim)
    print("probabilities calculated")
    
    #Reindex users for join
    reindexed_output = probabilities.reset_index().drop('index',axis=1).copy()
    indexed_unique_test = dataset.reset_index().drop('index',axis=1).copy()
    output_unique = indexed_unique_test.join(reindexed_output,rsuffix='_predict')
    output_final = pd.merge(test_remap,output_unique,on=used_columns,how='left')
    
    #Only select relevant products
    output_final = output_final.drop(used_columns,1)
    output_final.columns = output_final.columns.str.replace("_predict", "")
    output_final.columns = output_final.columns.str.replace("_previous", "_predict")
    
    #Have all test probabilities - can average and compare with results
    return output_final

In [116]:
#Step 6: Combine Demographic-Based and Memory-Based Probabilities

#Calculate memory-based similarities
probabilities_memory = probability_calculation(traindat_test_unique,traindat_train,traindat_purchases_train,new_product_col,'manhattan',traindat_test)

#Calculate demographic-based similarities
probabilities_demog = probability_calculation(traindat_demog_test_unique,traindat_demog_train,traindat_purchases_train,demog_col,'manhattan',traindat_demog_test)

#Average predictions for a range of mixing probabilities
probabilities_avg_95 = 0.95*probabilities_memory + 0.05*probabilities_demog
probabilities_avg_90 = 0.9*probabilities_memory + 0.1*probabilities_demog
probabilities_avg_85 = 0.85*probabilities_memory + 0.15*probabilities_demog
probabilities_avg_80 = 0.8*probabilities_memory + 0.2*probabilities_demog
probabilities_avg_75 = 0.75*probabilities_memory + 0.25*probabilities_demog
probabilities_avg_70 = 0.7*probabilities_memory + 0.3*probabilities_demog
probabilities_avg_65 = 0.65*probabilities_memory + 0.35*probabilities_demog
probabilities_avg_60 = 0.6*probabilities_memory + 0.4*probabilities_demog
probabilities_avg_55 = 0.55*probabilities_memory + 0.45*probabilities_demog
probabilities_avg_50 = 0.5*probabilities_memory + 0.5*probabilities_demog
probabilities_avg_45 = 0.45*probabilities_memory + 0.55*probabilities_demog
probabilities_avg_40 = 0.4*probabilities_memory + 0.6*probabilities_demog
probabilities_avg_35 = 0.35*probabilities_memory + 0.65*probabilities_demog
probabilities_avg_30 = 0.3*probabilities_memory + 0.7*probabilities_demog
probabilities_avg_25 = 0.25*probabilities_memory + 0.75*probabilities_demog
probabilities_avg_20 = 0.2*probabilities_memory + 0.8*probabilities_demog
probabilities_avg_15 = 0.15*probabilities_memory + 0.85*probabilities_demog
probabilities_avg_10 = 0.1*probabilities_memory + 0.9*probabilities_demog
probabilities_avg_5 = 0.05*probabilities_memory + 0.95*probabilities_demog


probabilities calculated


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3255830827.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  output_final = output_final.drop(used_columns,1)


probabilities calculated


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3255830827.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  output_final = output_final.drop(used_columns,1)


In [15]:
predict_col = [i + "_predict" for i in product_col]
predict_previous_col = predict_col + new_product_col

def purchase_nullifier(probabilities,purchase_history,print_option=False):
    #function to 'nullify' any probabilities that would lead to an owned product being predicted

    #Join two datasets together
    purchase_history = purchase_history.reset_index(drop=True)
    joined_data = purchase_history.join(probabilities)
    
    #Shrink dataset to deal with large-scale data
    unique_data = joined_data.drop_duplicates().copy().reset_index(drop=True)
    n = unique_data.shape[0]
    print("data joined")
    
    for index,row in unique_data.iterrows():
        if print_option == True:
            print(str(index) + "/" + str(n))
        row = row.to_frame().T
        #Subset dataframe and rename columns for nullification
        row_purchases = row[new_product_col]
        row_purchases.columns = row_purchases.columns.str.replace("_previous","")
        row_probabilities = row[predict_col]
        row_probabilities.columns = row_probabilities.columns.str.replace("_predict","")
        prob_norm = (1-row_purchases).multiply(row_probabilities,axis=0)
        if(index == 0):
            output_norm = prob_norm
        else:
            output_norm = output_norm.append(prob_norm)
    print("nullification complete")
    
    #Duplicate back up to original dataset
    #Add columns to enable merge
    output_index = output_norm.reset_index(drop=True)
    prob_predict = output_index.join(unique_data)
    scaled_predict = pd.merge(joined_data,prob_predict,how='left')
    output = scaled_predict[product_col]
    output.columns = output.columns.str.replace("ult1","ult1_predict")
    return output

In [117]:
nulled_probabilities_100 = purchase_nullifier(probabilities_memory,traindat_test)
nulled_probabilities_95 = purchase_nullifier(probabilities_avg_95,traindat_test)
nulled_probabilities_90 = purchase_nullifier(probabilities_avg_90,traindat_test)
nulled_probabilities_85 = purchase_nullifier(probabilities_avg_85,traindat_test)
nulled_probabilities_80 = purchase_nullifier(probabilities_avg_80,traindat_test)
nulled_probabilities_75 = purchase_nullifier(probabilities_avg_75,traindat_test)
nulled_probabilities_70 = purchase_nullifier(probabilities_avg_70,traindat_test)
nulled_probabilities_65 = purchase_nullifier(probabilities_avg_65,traindat_test)
nulled_probabilities_60 = purchase_nullifier(probabilities_avg_60,traindat_test)
nulled_probabilities_55 = purchase_nullifier(probabilities_avg_55,traindat_test)
nulled_probabilities_50 = purchase_nullifier(probabilities_avg_50,traindat_test)
nulled_probabilities_45 = purchase_nullifier(probabilities_avg_45,traindat_test)
nulled_probabilities_40 = purchase_nullifier(probabilities_avg_40,traindat_test)
nulled_probabilities_35 = purchase_nullifier(probabilities_avg_35,traindat_test)
nulled_probabilities_30 = purchase_nullifier(probabilities_avg_30,traindat_test)
nulled_probabilities_25 = purchase_nullifier(probabilities_avg_25,traindat_test)
nulled_probabilities_20 = purchase_nullifier(probabilities_avg_20,traindat_test)
nulled_probabilities_15 = purchase_nullifier(probabilities_avg_15,traindat_test)
nulled_probabilities_10 = purchase_nullifier(probabilities_avg_10,traindat_test)
nulled_probabilities_5 = purchase_nullifier(probabilities_avg_5,traindat_test)
nulled_probabilities_0 = purchase_nullifier(probabilities_demog,traindat_test)

data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete
data joined
nullification complete


In [107]:
#Step 7: Derive Recommendations

def probabilities_to_predictions(probabilities,ncodpers,print_option=False):
# ncodpers is a dataframe with two columns: fecha_dato and ncodpers (corresponding to probabilities order)    
    
    #Make probabilities unique to speed upc calculations
    unique_probabilities = probabilities.drop_duplicates().copy().reset_index(drop=True)
    print(unique_probabilities.shape)
    
    n = unique_probabilities.shape[0]
    
    for index, row in unique_probabilities.iterrows():
        if print_option == True:
            print(str(index) + '/' + str(n))
        row_use = row.to_frame().T
        
        #Rank list of product recommendations
        arank = row_use.apply(np.argsort, axis=1)
        ranked_cols = row_use.columns.to_series()[np. reshape(arank.values[:,::-1][:,:7],-1)] #Updated
        
        new_frame = pd.DataFrame(ranked_cols)
        
        #Concatenate all 7 predictions
        recoms = new_frame[0]
        recoms_final = recoms.str.replace('_predict', '', regex=True)
        if(index == 0):
            predictions = recoms_final
        else:
            predictions = predictions.append(recoms_final)
    
    #Merge predictions back to initial indices for full dataset
    mapped_predictions = predictions.to_frame().rename(columns={0:'added_products'}).reset_index(drop=True)
    output_unique = mapped_predictions.join(unique_probabilities)
    output_final = pd.merge(probabilities,output_unique,on=predict_col,how='left')
    
    #Add ncodpers for final submission file
    no_index_ncodpers = ncodpers.copy().reset_index(drop=True)
    output_ncodpers = no_index_ncodpers.join(output_final['added_products']).drop('fecha_dato',axis=1)
    return output_ncodpers

In [118]:
predictions_output_100 = probabilities_to_predictions(nulled_probabilities_100,traindat_test_ncodpers)
predictions_output_95 = probabilities_to_predictions(nulled_probabilities_95,traindat_test_ncodpers)
predictions_output_90 = probabilities_to_predictions(nulled_probabilities_90,traindat_test_ncodpers)
predictions_output_85 = probabilities_to_predictions(nulled_probabilities_85,traindat_test_ncodpers)
predictions_output_80 = probabilities_to_predictions(nulled_probabilities_80,traindat_test_ncodpers)
predictions_output_75 = probabilities_to_predictions(nulled_probabilities_75,traindat_test_ncodpers)
predictions_output_70 = probabilities_to_predictions(nulled_probabilities_70,traindat_test_ncodpers)
predictions_output_65 = probabilities_to_predictions(nulled_probabilities_65,traindat_test_ncodpers)
predictions_output_60 = probabilities_to_predictions(nulled_probabilities_60,traindat_test_ncodpers)
predictions_output_55 = probabilities_to_predictions(nulled_probabilities_55,traindat_test_ncodpers)
predictions_output_50 = probabilities_to_predictions(nulled_probabilities_50,traindat_test_ncodpers)
predictions_output_45 = probabilities_to_predictions(nulled_probabilities_45,traindat_test_ncodpers)
predictions_output_40 = probabilities_to_predictions(nulled_probabilities_40,traindat_test_ncodpers)
predictions_output_35 = probabilities_to_predictions(nulled_probabilities_35,traindat_test_ncodpers)
predictions_output_30 = probabilities_to_predictions(nulled_probabilities_30,traindat_test_ncodpers)
predictions_output_25 = probabilities_to_predictions(nulled_probabilities_25,traindat_test_ncodpers)
predictions_output_20 = probabilities_to_predictions(nulled_probabilities_20,traindat_test_ncodpers)
predictions_output_15 = probabilities_to_predictions(nulled_probabilities_15,traindat_test_ncodpers)
predictions_output_10 = probabilities_to_predictions(nulled_probabilities_10,traindat_test_ncodpers)
predictions_output_5 = probabilities_to_predictions(nulled_probabilities_5,traindat_test_ncodpers)
predictions_output_0 = probabilities_to_predictions(nulled_probabilities_0,traindat_test_ncodpers)

(2958, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6635, 24)
(6620, 24)


In [109]:
#Evaluation metric
#To ascertain the performance of each model (i.e. each mixing probability)

evaluation_col = product_col + ['added_products']

def evaluation_metric(predictions,reality,print_option=False):
    #Predictions is a list of the top seven purchase likelihood indicators; reality is the actual purchases
    reality = reality.reset_index(drop=True)
    
    #Find unique combinations to speed up function: merge data, group_by, count (then multiply results at the end)
    reality['added_products'] = predictions['added_products']
    data_unique = reality.drop_duplicates().copy().reset_index(drop=True)
    predictions_unique = data_unique['added_products'].to_frame()
    reality_unique = data_unique.drop('added_products',1)
    n = predictions_unique.shape[0]
    for index, row in predictions_unique.iterrows():
        if print_option == True:
            print(str(index) + '/' + str(n))
        prediction_use = row.to_frame().T['added_products'].str.split(' ',expand=True).T
        prediction_use = prediction_use.rename(columns={list(prediction_use)[0]:'predict_products'})

        #Only take top 7 products purchased
        reality_use = reality_unique.iloc[index].to_frame()
        reality_use = reality_use.rename(columns={list(reality_use)[0]:'added_products'})
        reality_use['product_name'] = reality_use.index
        reality_use = reality_use[reality_use.added_products==1]
        reality_use['ind'] = 1

        if reality_use.empty:
            P = [0]
        else:
            #Calculate precision @7: what average proportion of our predictions are purchased?
            P = [precision_at_k(prediction_use,reality_use)]
        if index == 0:
            eval_sum = P
        else:
            eval_sum.extend(P)
    
    #Duplicate back up
    print('precisions calculated')
    data_unique['precision'] = eval_sum
    reality_final = pd.merge(reality,data_unique,on=evaluation_col,how='left')
    U = predictions.shape[0]
    output = sum(reality_final.precision)/U
    return output

In [1]:
def precision_at_k(prediction,reality):
    #'prediction' is a data frame with a column 'predict_products' containing our 7 predictions
    #'reality' is a data frame with a column 'added_products' containing any products purchased (always non-empty)
    summand = min(prediction.shape[0],7)
    sum_prec = 0
    
    for k in range(summand):
        #Calculate precision at k (careful with 0 index)
        top_k_predictions = prediction.head(k+1)
        
        #Join additions to reduced predictions
        add_vs_pred = pd.merge(reality,top_k_predictions,left_on='product_name',right_on='predict_products',how='inner')
        sum_prec = sum_prec + sum(add_vs_pred.ind)/top_k_predictions.shape[0]
    
    denom = min(reality.shape[0],7)
    
    #Defined as in evaluation_metric function 'reality_use' is always non-empty
    output = sum_prec/denom
    
    return output 

In [119]:
evaluation_100 = evaluation_metric(predictions_output_100,traindat_purchases_test)
evaluation_95 = evaluation_metric(predictions_output_95,traindat_purchases_test)
evaluation_90 = evaluation_metric(predictions_output_90,traindat_purchases_test)
evaluation_85 = evaluation_metric(predictions_output_85,traindat_purchases_test)
evaluation_80 = evaluation_metric(predictions_output_80,traindat_purchases_test)
evaluation_75 = evaluation_metric(predictions_output_75,traindat_purchases_test)
evaluation_70 = evaluation_metric(predictions_output_70,traindat_purchases_test)
evaluation_65 = evaluation_metric(predictions_output_65,traindat_purchases_test)
evaluation_60 = evaluation_metric(predictions_output_60,traindat_purchases_test)
evaluation_55 = evaluation_metric(predictions_output_55,traindat_purchases_test)
evaluation_50 = evaluation_metric(predictions_output_50,traindat_purchases_test)
evaluation_45 = evaluation_metric(predictions_output_45,traindat_purchases_test)
evaluation_40 = evaluation_metric(predictions_output_40,traindat_purchases_test)
evaluation_35 = evaluation_metric(predictions_output_35,traindat_purchases_test)
evaluation_30 = evaluation_metric(predictions_output_30,traindat_purchases_test)
evaluation_25 = evaluation_metric(predictions_output_25,traindat_purchases_test)
evaluation_20 = evaluation_metric(predictions_output_20,traindat_purchases_test)
evaluation_15 = evaluation_metric(predictions_output_15,traindat_purchases_test)
evaluation_10 = evaluation_metric(predictions_output_10,traindat_purchases_test)
evaluation_5 = evaluation_metric(predictions_output_5,traindat_purchases_test)
evaluation_0 = evaluation_metric(predictions_output_0,traindat_purchases_test)

/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/2777252488.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  reality_unique = data_unique.drop('added_products',1)


precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated
precisions calculated


In [145]:
#[Other than modeling] Check the optimal weight between memory based and demographic based

#Specify the Column Names while initializing the Table 
myTable = PrettyTable(["Model", "Mixing Probability"]) 

model_list = ["All memory","95% memory","90% memory","85% memory", "80% memory","75% memory", "70% memory",
             "65% memory","60% memory","55% memory","50% memory","45% memory","40% memory","35% memory",
             "30% memory","25% memory","20% memory","15% memory","10% memory","5% memory","All demographics"]

memory_list = [evaluation_100, evaluation_95, evaluation_90, evaluation_85, evaluation_80, evaluation_75, 
              evaluation_70, evaluation_65, evaluation_60, evaluation_55, evaluation_50, evaluation_45, 
              evaluation_40, evaluation_35, evaluation_30, evaluation_25, evaluation_20, evaluation_15,
              evaluation_10, evaluation_5,evaluation_0]

for index in range(len(model_list)):
    myTable.add_row([model_list[index],memory_list[index]])
    
print(myTable)

max_value = max(memory_list)
max_model = model_list[memory_list.index(max_value)]

print("Maximum Mixing Probability: ", max_value, " (",max_model,")")

+------------------+-----------------------+
|      Model       |   Mixing Probability  |
+------------------+-----------------------+
|    All memory    |  0.003316739757417707 |
|    95% memory    |  0.004741021690174234 |
|    90% memory    |  0.004719292007427603 |
|    85% memory    |  0.004726535235009812 |
|    80% memory    |  0.004284039877260215 |
|    75% memory    |  0.004172099087353322 |
|    70% memory    |  0.004123371919982086 |
|    65% memory    |  0.004128639721860057 |
|    60% memory    |  0.004742338640643728 |
|    55% memory    |  0.004638299553553793 |
|    50% memory    |  0.004631714801206329 |
|    45% memory    |  0.004658712285830933 |
|    40% memory    |  0.004617886821276655 |
|    35% memory    |  0.004587596960478318 |
|    30% memory    |  0.004542820644515561 |
|    25% memory    |  0.004541503694046069 |
|    20% memory    |  0.004719950482662351 |
|    15% memory    |  0.004538211317872336 |
|    10% memory    | 0.0042122660766728505 |
|    5% me

In [146]:
#Step 8: Re-run model using all training data for optimal mixing parameter

#Calculate probabilities
probability_60_memory = probability_calculation(testdat_final_unique,traindat_final,traindat_purchases,new_product_col,'manhattan',testdat_final)
probability_60_demog = probability_calculation(testdat_demog_final_unique,traindat_demog_final,traindat_purchases,demog_col,'manhattan',testdat_demog_final)

#Average probabilities
probability_avg_60 = 0.6*probability_60_memory + 0.4*probability_60_demog

#Generate 1st csv file for averaged probabilities of each customer
probability_avg_60.to_csv("probabilities_60_avg.csv",index=False)

#Null previous ownership
nulled_probability_60 = purchase_nullifier(probability_avg_60[predict_col],testdat_final[new_product_col])

#Map to predictions - check dimensions
testdat_ncodpers = testdat_use[['fecha_dato','ncodpers']]
predictions_output_60 = probabilities_to_predictions(nulled_probability_60,testdat_ncodpers)

#Generate 2nd csv file for Top 1 recommended product of each customer
predictions_output_60.to_csv('Final Recommendation Result Per Customer.csv',index=False)

probabilities calculated


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3255830827.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  output_final = output_final.drop(used_columns,1)


probabilities calculated


/var/folders/0p/wklr8kqj0h52vzxckdpz9xhc0000gn/T/ipykernel_716/3255830827.py:32: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  output_final = output_final.drop(used_columns,1)


data joined
nullification complete
(15419, 24)


In [1]:
#[Other than modeling] Split the large probability file into multiple files with smaller size in order to upload the result files to Github

csvfile = open('probabilities_60_avg.csv', 'r').readlines()
filename = "probabilities_60_avg_"
filecount = 1

for i in range(len(csvfile)):
    if i % 30000 == 0:
        fullfilename = filename + str(filecount)
        open(fullfilename + '.csv', 'w+').writelines(csvfile[i:i+30000])
        filecount += 1
        print("Generated " + fullfilename)

Generated probabilities_60_avg_1
Generated probabilities_60_avg_2
Generated probabilities_60_avg_3
Generated probabilities_60_avg_4
Generated probabilities_60_avg_5
Generated probabilities_60_avg_6
Generated probabilities_60_avg_7
Generated probabilities_60_avg_8
Generated probabilities_60_avg_9
Generated probabilities_60_avg_10
Generated probabilities_60_avg_11
Generated probabilities_60_avg_12
Generated probabilities_60_avg_13
Generated probabilities_60_avg_14
Generated probabilities_60_avg_15
Generated probabilities_60_avg_16
Generated probabilities_60_avg_17
Generated probabilities_60_avg_18
Generated probabilities_60_avg_19
Generated probabilities_60_avg_20
Generated probabilities_60_avg_21
Generated probabilities_60_avg_22
Generated probabilities_60_avg_23
Generated probabilities_60_avg_24
Generated probabilities_60_avg_25
Generated probabilities_60_avg_26
Generated probabilities_60_avg_27
Generated probabilities_60_avg_28
Generated probabilities_60_avg_29
Generated probabilities